In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten, LeakyReLU, BatchNormalization

In [ ]:
train = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')
IdLookupTable = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
SampleSubmission = pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')

In [ ]:
features = list(train.columns)
train.head().T

In [ ]:
train = train.fillna(method='ffill')

In [ ]:
Y = np.array(train.iloc[:,0:-1])
Y.shape

In [ ]:
train = np.array([np.array(im_str.split(' '), dtype=np.uint8) for im_str in train.iloc[:,-1]])
train = train.reshape([7049,96,96,1])
train.shape

In [ ]:
def plot_keypoints(img, keypoints):
    plt.imshow(img.reshape([96,96]), cmap='gray')
    for i in range(0,30,2):
        plt.plot(int(round(keypoints[i])),int(round(keypoints[i+1])), 'ro')
    plt.show()

plot_keypoints(train[0], Y[0])

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=(96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3,3), padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3,3),padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3,3),padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 0.003, decay_steps=5500, decay_rate=0.5, staircase=False)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='mse', optimizer=opt, metrics=['mae'])
model.fit(train, Y, batch_size=16, epochs=80, verbose=1, validation_split=0.2, shuffle=True)

In [ ]:
test = np.array([np.array(im_str.split(' '), dtype=np.uint8) for im_str in test.iloc[:,-1]])
test = test.reshape([-1,96,96,1])
preds = model.predict(test)

In [ ]:
im = test[0]
keypoints = model.predict(im.reshape([-1,96,96,1]))[0]
plot_keypoints(im, keypoints)

In [ ]:
submission = pd.DataFrame()
submission['RowId'] = IdLookupTable['RowId']
submission['Location'] = pd.Series([preds[row['ImageId']-1, features.index(row['FeatureName'])] \
                                    for i,row in IdLookupTable.iterrows()])

In [ ]:
submission.to_csv('facial_keypoint_detection_submission2.csv',index = False)